
This practical uses the file `flight_info.csv`, which you can download from [this link](https://www.dropbox.com/scl/fi/jsbatbyh5hl8q187986v1/flight_info.csv?rlkey=8ky753wksm5e6zbxdhvcoz2c9&dl=1).

**Important**: You must use Apache Spark for all tasks, even though this can technically run on a single machine. The goal is to practice using Spark in a way that simulates a distributed computing environment, so avoid any approach that wouldn't run on a single machine if the dataset was substantially larger. 

1. Read the file into a Spark DataFrame named `df`.
   * Ensure the schema is inferred automatically, and indicate that the file includes a header row.

2. Use any Spark method of your choice to display the first 10 rows of the DataFrame.

3. How many unique carriers are listed in the file?

4. How many flights are associated with each carrier?

5. Which hour of the day has the most flight take offs?



1. Read the file into a Spark DataFrame named `df`.
   * Ensure the schema is inferred automatically, and indicate that the file includes a header row.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("flight_info").getOrCreate()
df = spark.read.format("csv").option("header", "true").load("./flight_info.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/03 12:37:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2. Use any Spark method of your choice to display the first 10 rows of the DataFrame.


In [2]:
df.show(10)

24/10/03 12:37:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
 Schema: _c0, DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
Expected: _c0 but found: 
CSV file: file:///Users/sebastiansole/Documents/NTNU/9sem/bigdata/ics-438-assignments/e05/flight_info.csv


+---+---------+-------------+---------+------+----+----------+-------+-------+---------+--------+------+----------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|DayOfWeek|UniqueCarrier|FlightNum|Origin|Dest|CRSDepTime|DepTime|TaxiOut|WheelsOff|WheelsOn|TaxiIn|CRSArrTime|ArrTime|Cancelled|CancellationCode|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+---------+-------------+---------+------+----+----------+-------+-------+---------+--------+------+----------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|        2|           AA|      494|   CLT| PHX|      1619| 1616.0|   17.0|   1633.0|  1837.0|   5.0|      1856| 1842.0|      0.0|            NULL|  1773.0|        NULL|        NULL|    NULL|         NULL|             NULL|
|  1|        3|           AA|      494|   CLT| PHX|      1619| 1614.0|   13.0|   1627.0|  18

3. How many unique carriers are listed in the file?

In [3]:
# Testing without RDD
unique_carriers = df.select("UniqueCarrier").distinct().count()

df.select("UniqueCarrier").distinct().show()
print("Total number of unique carriers: ", unique_carriers)


+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|           EV|
|           B6|
|           DL|
|           UA|
|           NK|
|           OO|
|           F9|
|           HA|
|           WN|
|           AS|
|           VX|
+-------------+

Total number of unique carriers:  12


In [4]:
# Creating a RDD
df_rdd = df.rdd

In [5]:
# Testing with RDD
unique_carriers_rdd = df_rdd.map(lambda row: (row['UniqueCarrier'], 1)).reduceByKey(lambda a, b: a + b)
collect = unique_carriers_rdd.collect()
unique_carriers_count = unique_carriers_rdd.count()

print("Total number of unique carriers: ", unique_carriers_count)
print(collect)

24/10/03 12:37:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
 Schema: _c0, DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
Expected: _c0 but found: 
CSV file: file:///Users/sebastiansole/Documents/NTNU/9sem/bigdata/ics-438-assignments/e05/flight_info.csv


Total number of unique carriers:  12
[('DL', 69813), ('UA', 42403), ('AA', 73132), ('HA', 6276), ('B6', 24602), ('EV', 35037), ('AS', 14711), ('VX', 5782), ('F9', 7760), ('NK', 12570), ('OO', 50146), ('WN', 107785)]


# 4. How many flights were there from each carrier? (Hint: Use wheel off)


In [6]:
flight_takeoffs_rdd = df_rdd.filter(lambda flight: flight['WheelsOff'])

In [7]:
flight_takeoffs_rdd_mapped = flight_takeoffs_rdd.map(lambda flight: (flight['UniqueCarrier'], 1)).reduceByKey(lambda a, b: a + b)
flight_takeoffs_rdd_mapped.collect()

24/10/03 12:37:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
 Schema: _c0, DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
Expected: _c0 but found: 
CSV file: file:///Users/sebastiansole/Documents/NTNU/9sem/bigdata/ics-438-assignments/e05/flight_info.csv


[('DL', 69044),
 ('UA', 42174),
 ('AA', 72159),
 ('HA', 6258),
 ('B6', 24082),
 ('EV', 33890),
 ('AS', 14458),
 ('VX', 5645),
 ('F9', 7598),
 ('NK', 12133),
 ('OO', 48324),
 ('WN', 105479)]

# 5. Which hour of the day has the most flight take offs?


In [8]:
flight_times = flight_takeoffs_rdd.map(lambda flight: (flight['CRSDepTime'][:-2], 1)).reduceByKey(lambda a, b: a + b)
flight_times_formatted = flight_times.map(lambda flight: ('0' + flight[0], flight[1]) if flight[0] == '' else flight)
flights_sorted = flight_times_formatted.sortBy(lambda flight: int(flight[0]))
flights_sorted.collect()

24/10/03 12:37:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
 Schema: _c0, DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
Expected: _c0 but found: 
CSV file: file:///Users/sebastiansole/Documents/NTNU/9sem/bigdata/ics-438-assignments/e05/flight_info.csv


[('0', 1045),
 ('1', 372),
 ('2', 215),
 ('3', 126),
 ('4', 86),
 ('5', 9224),
 ('6', 28945),
 ('7', 30262),
 ('8', 29952),
 ('9', 27649),
 ('10', 26924),
 ('11', 28343),
 ('12', 26958),
 ('13', 27496),
 ('14', 25611),
 ('15', 28252),
 ('16', 25795),
 ('17', 29886),
 ('18', 25178),
 ('19', 24025),
 ('20', 19877),
 ('21', 12279),
 ('22', 9264),
 ('23', 3480)]